In [2]:
! pip install -q tensorflow-model-optimization

You should consider upgrading via the 'c:\users\jainy\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
keras.layers.InputLayer(input_shape=(28, 28)),
keras.layers.Reshape(target_shape=(28, 28, 1)),
    
keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.4),

keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.4),

keras.layers.Flatten(),

keras.layers.Dense(10, activation='relu'),
#keras.layers(layers.Dense(units=128, activation='relu'))
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.4),


keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 33s 20ms/step - loss: 0.7484 - accuracy: 0.7604 - val_loss: 0.1111 - val_accuracy: 0.9738
Epoch 2/4
1688/1688 [==============================] - 33s 20ms/step - loss: 0.3826 - accuracy: 0.8716 - val_loss: 0.0589 - val_accuracy: 0.9838
Epoch 3/4
1688/1688 [==============================] - 34s 20ms/step - loss: 0.3349 - accuracy: 0.8853 - val_loss: 0.0608 - val_accuracy: 0.9828
Epoch 4/4
1688/1688 [==============================] - 35s 21ms/step - loss: 0.3060 - accuracy: 0.8949 - val_loss: 0.0574 - val_accuracy: 0.9842


Evaluate baseline test accuracy and save the model for later usage.

In [3]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9811999797821045
Saved baseline model to: C:\Users\JAINY\AppData\Local\Temp\tmpb3mr2cbn.h5


## Fine-tune pre-trained model with pruning


In [4]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 128
epochs = 3
validation_split = 0.1  

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_batch_no (None, 13, 13, 12)        49        
_________________________________________________________________
prune_low_magnitude_dropout  (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 11, 11, 64)        13890     
_________________________________________________

### Train and evaluate the model against baseline

In [5]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9811999797821045
Pruned test accuracy: 0.9811999797821045


# Using strip pruning from tesorflow. This apply compression algorithm to further reduce the size.

In [6]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: C:\Users\JAINY\AppData\Local\Temp\tmpqn7y0ueq.h5


In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\JAINY\AppData\Local\Temp\tmp8tfeltjz\assets
Saved pruned TFLite model to: C:\Users\JAINY\AppData\Local\Temp\tmp4cawv71t.tflite


In [8]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [9]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 91672.00 bytes
Size of gzipped pruned Keras model: 91673.00 bytes
Size of gzipped pruned TFlite model: 89616.00 bytes


# using quantization along with pruning from keras tensorflow

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: C:\Users\JAINY\AppData\Local\Temp\tmp8agxolrg\assets


INFO:tensorflow:Assets written to: C:\Users\JAINY\AppData\Local\Temp\tmp8agxolrg\assets


Saved quantized and pruned TFLite model to: C:\Users\JAINY\AppData\Local\Temp\tmp0avn8mc4.tflite
Size of gzipped baseline Keras model: 91672.00 bytes
Size of gzipped pruned and quantized TFlite model: 24493.00 bytes


In [12]:
import numpy as np

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    prediction_digits = []
    for i, test_image in enumerate(test_images):
        if i % 1000 == 0:
            print('Evaluated on {n} results so far.'.format(n=i))
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    interpreter.invoke()

    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [13]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.0958
Pruned TF test accuracy: 0.9811999797821045
